In [77]:
#!pip install texthero
import pandas as pd
import numpy as np
import texthero as hero
from texthero import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [2]:
def readin(file):
    return pd.read_csv(f"Data/{file}.csv", index_col=0)

In [3]:
df_b_r = readin('df_books_rev')

In [4]:
df_b_r

,asin,reviewText
0,B0012GTZCK,Not what I was expecting. I was looking for a ...
1,B0012LHGJ4,IT WAS DONE AS A FILM IN THE 1950-60s. VERY P...
2,B0012RMVH0,Needs pictures Excellent book. This is somethi...
3,B0012TAD1O,She always delivers with an exciting storyline...
4,B0012U0NKE,"<a data-hook=""product-link-linked"" class=""a-li..."
...,...,...
93542,B01HJA2J0C,"Get rid of a few errors, and it's a good book...."
93543,B01HJBPMQY,"I thought this was a quick, fun read. Great de..."
93544,B01HJBPUWA,I once stuck my head inside my mother's purse ...
93545,B01HJC63YI,This has to be read...an outstanding series Ou...


In [142]:
#df_r_r = readin('df_reviewers_rev')

In [143]:
#df_r_r.head()

,reviewerID,reviewText
0,A00463782V7TKAP9EMNL,College roommates find love for each other at ...
1,A0092581WFYQNV4KMUZ3,I guess the book was ok it just wasn't what I ...
2,A0099735VDZ3HDCAAYKL,I really hated that this was the last story it...
3,A01423305PUDQS2VV7QX,I really enjoyed this book. Lots of turns & su...
4,A01433701FZ0LF5G2RUL,I can't read anything about children especiall...


In [144]:
#df_b_s = readin('df_books_sum')

In [145]:
#df_b_s

,asin,summary
0,B0012GTZCK,Not what I anticipated A must read
1,B0012LHGJ4,MOVIE MATERIAL MOVIE MATERIAL
2,B0012RMVH0,One Star Excellent book. This is something I h...
3,B0012TAD1O,Another great story from Kallysten! Another gr...
4,B0012U0NKE,Good read...
...,...,...
93542,B01HJA2J0C,Good
93543,B01HJBPMQY,fun read. Great description Hello Tarzan! Five...
93544,B01HJBPUWA,"Ahhhhhh the wonders of my childhood, relived s..."
93545,B01HJC63YI,WOW!! OUTSTANDING! Good read The BEST... Inten...


In [146]:
#df_r_s = readin('df_reviewers_sum')

In [147]:
#df_r_s

,reviewerID,summary
0,A00463782V7TKAP9EMNL,Loved the Roommates Simply Wonderful iIs this ...
1,A0092581WFYQNV4KMUZ3,Connections Delilah Diary #3 glass Wanting mor...
2,A0099735VDZ3HDCAAYKL,What a wild story Short and sweet A good book ...
3,A01423305PUDQS2VV7QX,Great Read Great Read Great Read Great Read Gr...
4,A01433701FZ0LF5G2RUL,Embryo Edwina Departed Trust Peek A boo I See...
...,...,...
97583,AZZUXD2N2CUDM,What happened to Tommy and Grace Book Four She...
97584,AZZV9PDNMCOZW,Cute A Quick Read Another good Aunt Bessie rea...
97585,AZZVMAAPAZU6D,Four Stars Four Stars Four Stars Five Stars Fo...
97586,AZZW5NCOD8IA0,Okay book for basics. Another winner by Mark F...


In [5]:
def clean_review(df):
    custom_pipeline = [preprocessing.lowercase,
                  preprocessing.remove_punctuation,
                  preprocessing.remove_stopwords,
                  preprocessing.stem,
                  preprocessing.remove_whitespace]
    df['clean_text'] = hero.clean(df['reviewText'], custom_pipeline)

In [149]:
def clean_summary(df):
    custom_pipeline = [preprocessing.lowercase,
                  preprocessing.remove_punctuation,
                  preprocessing.remove_stopwords,
                  preprocessing.stem,
                  preprocessing.remove_whitespace]
    df['clean_text'] = hero.clean(df['summary'], custom_pipeline)

In [6]:
clean_review(df_b_r)

In [8]:
df_b_r.set_index('asin', inplace=True)

In [9]:
df_b_r.head()

,reviewText,clean_text
asin,,
B0012GTZCK,Not what I was expecting. I was looking for a ...,expect look histori christian influenc live pe...
B0012LHGJ4,IT WAS DONE AS A FILM IN THE 1950-60s. VERY P...,done film 1950 60s popular film back today kid...
B0012RMVH0,Needs pictures Excellent book. This is somethi...,need pictur excel book someth want book give k...
B0012TAD1O,She always delivers with an exciting storyline...,alway deliv excit storylin lot heat cours imag...
B0012U0NKE,"<a data-hook=""product-link-linked"" class=""a-li...",data hook product link link class link normal ...


In [63]:
tf = TfidfVectorizer(max_features=300)
dtm = tf.fit_transform(df_b_r['clean_text'])
dtm = pd.DataFrame(dtm.todense(), columns=tf.get_feature_names(), index=df_b_r.index)

In [64]:
dtm

,abl,absolut,action,actual,adventur,almost,along,alreadi,also,although,...,world,worth,would,write,writer,written,wrong,year,yet,young
asin,,,,,,,,,,,,,,,,,,,,,
B0012GTZCK,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000
B0012LHGJ4,0.000000,0.0,0.000000,0.000000,0.362567,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.184134,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.326625
B0012RMVH0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.10009,0.000000,0.000000
B0012TAD1O,0.000000,0.0,0.090734,0.091607,0.000000,0.000000,0.000000,0.0,0.000000,0.099197,...,0.085777,0.000000,0.110961,0.067045,0.200192,0.065917,0.0,0.00000,0.000000,0.000000
B0012U0NKE,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B01HJA2J0C,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.513400,0.000000
B01HJBPMQY,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.201423,0.128743,0.000000,0.000000,0.152960,0.0,0.00000,0.000000,0.000000
B01HJBPUWA,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.047105,0.0,0.098276,0.245137,...,0.211973,0.000000,0.000000,0.000000,0.049472,0.065158,0.0,0.00000,0.090621,0.048640


In [70]:
df_meta = pd.read_csv('Data/meta.csv', index_col='asin')
df_meta.drop(columns =['Unnamed: 0'], inplace=True)

In [71]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93547 entries, B0012GTZCK to B01HJCNGZ2
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         93547 non-null  object 
 1   brand         93547 non-null  object 
 2   genre         93547 non-null  object 
 3   print_length  93547 non-null  float64
 4   word_wise     93547 non-null  int64  
 5   lending       93547 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 5.0+ MB


In [89]:
df_meta.head()

,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B0012GTZCK,God in the White House,Randall Balmer,History,260.0,1,0
B0012LHGJ4,Vingt Mille Lieues sous les mers (French Editi...,Jules Verne,Reference,606.0,0,1
B0012RMVH0,Wood-Carving Design and Workmanship - Kindle e...,George Jack,Arts & Photography,310.0,0,0
B0012TAD1O,Souls Night (The Vampire Pacts) - Kindle edition,Kallysten,Romance,74.0,1,1
B0012U0NKE,The Misplaced Horse (1) eBook,Constance Downes,"Comics, Manga & Graphic Novels",444.0,0,1


In [93]:
model_df = dtm.merge(df_meta, left_index=True, right_index=True)

In [94]:
model_df.drop(columns=['title', 'brand', 'word_wise', 'lending'], inplace=True)

In [95]:
model_df = pd.get_dummies(model_df, columns=['genre'])

In [96]:
model_df

,abl,absolut,action,actual,adventur,almost,along,alreadi,also,although,...,genre_Politics & Social Sciences,genre_Reference,genre_Religion & Spirituality,genre_Romance,genre_Science & Math,genre_Science Fiction & Fantasy,genre_Self-Help,genre_Sports & Outdoors,genre_Teen & Young Adult,genre_Travel
asin,,,,,,,,,,,,,,,,,,,,,
B0012GTZCK,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
B0012LHGJ4,0.000000,0.0,0.000000,0.000000,0.362567,0.000000,0.000000,0.0,0.000000,0.000000,...,0,1,0,0,0,0,0,0,0,0
B0012RMVH0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
B0012TAD1O,0.000000,0.0,0.090734,0.091607,0.000000,0.000000,0.000000,0.0,0.000000,0.099197,...,0,0,0,1,0,0,0,0,0,0
B0012U0NKE,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B01HJA2J0C,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
B01HJBPMQY,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
B01HJBPUWA,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.047105,0.0,0.098276,0.245137,...,0,0,0,0,0,1,0,0,0,0


In [97]:
cols = ['print_length']
scaler = MinMaxScaler()
scaled = scaler.fit_transform(model_df[cols])
scaled_column = pd.DataFrame(scaled, index=model_df.index, columns=cols)
model_df['print_length'] = scaled_column['print_length']

In [98]:
y = np.array(model_df.loc['B01HJBPUWA'])
y = y.reshape(1, -1)
cos_sim = cosine_similarity(model_df, y)
cos_sim = pd.DataFrame(data=cos_sim, index=model_df.index)
cos_sim.sort_values(by = 0, ascending=False)

,0
asin,
B01HJBPUWA,1.000000e+00
B01DE64NKE,8.371681e-01
B00B0071TC,8.357615e-01
B00C1MZN5C,8.349038e-01
B00NGANDX2,8.321457e-01
...,...
B01CXCTJFO,2.508696e-06
B00KWS0VB8,2.418976e-06
B007HAES2C,2.096446e-06
